In [7]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [8]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [9]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [10]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [15]:
X = X.loc[:, [col for col in X if not col.startswith('Soil_Type')]]

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# Define the base estimator for AdaBoost with a more complex decision tree
base_estimator = DecisionTreeClassifier()  

# Define the AdaBoost classifier using the base estimator
estimator = AdaBoostClassifier(estimator=base_estimator, algorithm='SAMME')

# Define hyperparameters for tuning both AdaBoost and its base estimator
hyperparameters = {
    "estimator__criterion": ["gini", "entropy"],
    "estimator__splitter": ["best", "random"],
    "estimator__max_features": [None, "sqrt", "log2"],
    "estimator__max_depth": [None, 2, 3, 4, 5, 7, 10, 12, 15],  
    "n_estimators": stats.randint(50, 500),
    "learning_rate": stats.uniform(0.01, 1.0),
}


random_search = RandomizedSearchCV(estimator,
                                   param_distributions=hyperparameters,
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500,
                                   cv=5,
                                   verbose=10,
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_ada/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"ada_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
    
    # random_search = pickle.load(open(file_name, "rb"))
    

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 2/5; 2/500] START estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.6964825982338081, n_estimators=319
[CV 2/5; 2/500] END estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.6964825982338081, n_estimators=319;, score=(train=0.664, test=0.649) total time=   3.2s
[CV 4/5; 2/500] START estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.6964825982338081, n_estimators=319
[CV 4/5; 2/500] END estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.6964825982338081, n_estimators=319;, score=(train=0.680, test=0.666) total time=   3.0s
[CV 3/5; 3/500] START estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=sqrt, estimat

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.


[CV 4/5; 1/500] START estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422
[CV 4/5; 1/500] END estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422;, score=(train=1.000, test=0.791) total time=   6.9s
[CV 1/5; 4/500] START estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123
[CV 1/5; 4/500] END estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123;, score=(train=0.826, test=0.778) total time=  20.0s
[CV 5/5; 6/500] START estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.23711245016223115,

[CV 1/5; 1/500] START estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422
[CV 1/5; 1/500] END estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422;, score=(train=1.000, test=0.791) total time=   7.1s
[CV 3/5; 4/500] START estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123
[CV 3/5; 4/500] END estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123;, score=(train=0.833, test=0.784) total time=  19.8s
[CV 4/5; 6/500] START estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.23711245016223115,

[CV 3/5; 1/500] START estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422
[CV 3/5; 1/500] END estimator__criterion=gini, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=1.0096343640854035, n_estimators=422;, score=(train=1.000, test=0.788) total time=   7.0s
[CV 2/5; 4/500] START estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123
[CV 2/5; 4/500] END estimator__criterion=entropy, estimator__max_depth=5, estimator__max_features=None, estimator__splitter=best, learning_rate=0.1623579062189605, n_estimators=123;, score=(train=0.837, test=0.791) total time=  20.3s
[CV 1/5; 7/500] START estimator__criterion=gini, estimator__max_depth=4, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.585503337191054, n_es

...

[CV 4/5; 383/500] END estimator__criterion=gini, estimator__max_depth=5, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.5942234638776089, n_estimators=335;, score=(train=0.853, test=0.764) total time=   6.9s
[CV 4/5; 384/500] START estimator__criterion=entropy, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.7647089656899251, n_estimators=103
[CV 4/5; 384/500] END estimator__criterion=entropy, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.7647089656899251, n_estimators=103;, score=(train=0.659, test=0.638) total time=   1.5s
[CV 2/5; 385/500] START estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.8704475146364014, n_estimators=302
[CV 2/5; 385/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.8704475

[CV 5/5; 384/500] END estimator__criterion=entropy, estimator__max_depth=2, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.7647089656899251, n_estimators=103;, score=(train=0.667, test=0.643) total time=   1.6s
[CV 3/5; 385/500] START estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.8704475146364014, n_estimators=302
[CV 3/5; 385/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.8704475146364014, n_estimators=302;, score=(train=1.000, test=0.800) total time=   6.0s
[CV 1/5; 386/500] START estimator__criterion=gini, estimator__max_depth=4, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.04731396197743109, n_estimators=103
[CV 1/5; 386/500] END estimator__criterion=gini, estimator__max_depth=4, estimator__max_features=log2, estimator__splitter=random, learning_rate=0.04731

[CV 1/5; 378/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=None, estimator__splitter=random, learning_rate=0.8477356863885122, n_estimators=471;, score=(train=1.000, test=0.843) total time=  30.2s
[CV 4/5; 380/500] START estimator__criterion=entropy, estimator__max_depth=10, estimator__max_features=None, estimator__splitter=best, learning_rate=0.7962448563996908, n_estimators=304
[CV 4/5; 380/500] END estimator__criterion=entropy, estimator__max_depth=10, estimator__max_features=None, estimator__splitter=best, learning_rate=0.7962448563996908, n_estimators=304;, score=(train=1.000, test=0.883) total time= 2.3min
[CV 3/5; 396/500] START estimator__criterion=entropy, estimator__max_depth=3, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.19529354623992956, n_estimators=448
[CV 3/5; 396/500] END estimator__criterion=entropy, estimator__max_depth=3, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.

[CV 2/5; 430/500] END estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=None, estimator__splitter=best, learning_rate=0.7373439133788138, n_estimators=207;, score=(train=0.606, test=0.594) total time=  11.3s
[CV 5/5; 430/500] START estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=None, estimator__splitter=best, learning_rate=0.7373439133788138, n_estimators=207
[CV 5/5; 430/500] END estimator__criterion=gini, estimator__max_depth=2, estimator__max_features=None, estimator__splitter=best, learning_rate=0.7373439133788138, n_estimators=207;, score=(train=0.592, test=0.581) total time=  11.2s
[CV 2/5; 432/500] START estimator__criterion=gini, estimator__max_depth=15, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.8093967373834883, n_estimators=245
[CV 2/5; 432/500] END estimator__criterion=gini, estimator__max_depth=15, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.8093967373834883, n_est

[CV 2/5; 413/500] END estimator__criterion=entropy, estimator__max_depth=4, estimator__max_features=sqrt, estimator__splitter=best, learning_rate=0.8432997727438081, n_estimators=101;, score=(train=0.710, test=0.678) total time=   3.0s
[CV 5/5; 414/500] START estimator__criterion=gini, estimator__max_depth=15, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.33154601130966366, n_estimators=433
[CV 5/5; 414/500] END estimator__criterion=gini, estimator__max_depth=15, estimator__max_features=sqrt, estimator__splitter=random, learning_rate=0.33154601130966366, n_estimators=433;, score=(train=1.000, test=0.800) total time=  11.0s
[CV 1/5; 418/500] START estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.7987043051135079, n_estimators=458
[CV 1/5; 418/500] END estimator__criterion=entropy, estimator__max_depth=12, estimator__max_features=log2, estimator__splitter=best, learning_rate=0.798704

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_ada/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Validation Score")

plt.grid()
plt.xlabel('Sorted Validation Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train set, Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Test  set, Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(f'./tuning_results/tuning_ada/{timestamp}/', file_name), "rb"))
model = random_search.best_estimator_

#model = ADAClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance ADA')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In [ ]:
# TEST
for col in ['param_estimator__splitter', 'param_estimator__criterion',
       'param_estimator__splitter', 'param_estimator__max_features', 'param_estimator__max_depth',
       'param_n_estimators', 'param_learning_rate']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['mean_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['mean_test_score'], m * sorted_cv['mean_test_score'] + b, c='r', label="Regression Line")
    plt.scatter(sorted_cv['mean_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Mean Validation Score')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    #ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r', label='std_test_score')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:

for param in ['param_estimator__splitter', 'param_estimator__criterion',
       'param_estimator__splitter', 'param_estimator__max_features', 'param_estimator__max_depth',
       'param_n_estimators', 'param_learning_rate']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


# KAGGLE Prediction

In [ ]:
test_processed = pd.read_csv('Data/test_engineered.csv')

In [ ]:
test_processed.head()

In [ ]:
test_processed = test_processed.loc[:, [col for col in test_processed if not col.startswith('Cover_Type_')]]
X_kaggle = test_processed.drop(columns=['Aspect_Sector'])
y_kaggle = model.predict(X_kaggle) + 1

In [ ]:
pd.read_csv("Data/Kaggle/full_submission.csv").head()

In [ ]:
test_processed['Cover_Type'] = y_kaggle

In [ ]:
kaggle_submission = test_processed.loc[:, ['Id', 'Cover_Type']]

In [ ]:
kaggle_submission.Cover_Type.value_counts()

In [ ]:
kaggle_submission.to_csv(f'Data/kaggle_submission_{timestamp}.csv', index=False)

In [ ]:
pd.read_csv(f'Data/kaggle_submission_{timestamp}.csv')